In [1]:
import numpy as np
import pandas as pd

In [2]:
def add_year_month(df, time_col='time'):
    dt = pd.to_datetime(df[time_col])
    df['year'] = dt.dt.year
    df['month'] = dt.dt.month
    return df

In [3]:
seass = pd.read_csv("output/SEASS_midwest_T2.csv")
seass = add_year_month(seass)
obs = pd.read_csv("output/OBS_midwest_T2.csv")
obs = add_year_month(obs)

In [4]:
def compute_anomaly_corr(seass, obs, vars_to_process=["T2MAX", "T2MIN"]):
    """
    Compute anomaly correlations between forecast and observed data for specified variables.

    Parameters
    ----------
    seass : pd.DataFrame
        Forecast dataset with columns ['start_month', 'lead_time', 'number', 'year', 'month', vars_to_process...]
    obs : pd.DataFrame
        Observational dataset with columns ['year', 'month', vars_to_process...]
    vars_to_process : list of str
        Variables for which to compute anomaly correlations (default: ['T2MAX','T2MIN'])

    Returns
    -------
    pd.DataFrame
        DataFrame with columns ['start_month','lead_time','ensemble','variable','correlation']
    """
    results = []

    # Loop over all forecast start months
    for sm in sorted(seass['start_month'].unique()):
        df_sm = seass[seass['start_month'] == sm]

        # Loop over all forecast lead times
        for lt in sorted(df_sm['lead_time'].unique()):
            df_lt = df_sm[df_sm['lead_time'] == lt]

            # Loop over all ensemble members
            for ens in sorted(df_lt['number'].unique()):
                df_ens = df_lt[df_lt['number'] == ens]

                # Loop over each variable of interest
                for var in vars_to_process:

                    # Identify overlapping years between forecast and observations
                    common_years = np.intersect1d(df_ens['year'].unique(), obs['year'].unique())

                    # TODO For this proof-of-concept, force the analysis to last 5 years
                    common_years = [2020, 2021, 2022, 2023, 2024]
                    if len(common_years) == 0:
                        continue

                    # Subset forecast and observation data to the selected years
                    fcst = df_ens[df_ens['year'].isin(common_years)][['year','month',var]].copy()
                    obsv = obs[obs['year'].isin(common_years)][['year','month',var]].copy()

                    # Compute monthly climatology for each variable
                    clim_fcst = fcst.groupby('month')[var].transform('mean')
                    clim_obs  = obsv.groupby('month')[var].transform('mean')

                    # Calculate anomalies by subtracting monthly climatology
                    fcst['anom'] = fcst[var] - clim_fcst
                    obsv['anom'] = obsv[var] - clim_obs

                    # Align forecast and observation anomalies by year and month
                    merged = pd.merge(fcst, obsv, on=['year','month'], suffixes=('_fcst','_obs'))

                    # Compute correlation of anomalies across years
                    if len(merged) > 1:
                        corr = merged['anom_fcst'].corr(merged['anom_obs'])
                    else:
                        corr = np.nan

                    # Store result for this combination of start_month, lead_time, ensemble, and variable
                    results.append({
                        'start_month': sm,
                        'lead_time': lt,
                        'ensemble': ens,
                        'variable': var,
                        'correlation': corr
                    })

    # Return all results as a DataFrame
    return pd.DataFrame(results)


In [8]:
anomaly_corr = compute_anomaly_corr(seass, obs, vars_to_process=["T2MAX", "T2MIN"])
anomaly_corr.to_csv("output/anomaly_corr.csv", index=False)
anomaly_corr

,start_month,lead_time,ensemble,variable,correlation
0,1,0,0,T2MAX,-0.313361
1,1,0,0,T2MIN,-0.067518
2,1,0,1,T2MAX,-0.257885
3,1,0,1,T2MIN,0.113398
4,1,0,2,T2MAX,0.339319
...,...,...,...,...,...
5503,9,5,48,T2MIN,0.267736
5504,9,5,49,T2MAX,-0.260775
5505,9,5,49,T2MIN,-0.050953
5506,9,5,50,T2MAX,0.221388


In [6]:
def summarize_with_brackets(df, percentiles=[33, 50, 66]):
    """
    Summarize ensemble correlations by percentiles and classify ensemble member skill.

    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe with columns:
        ['start_month', 'lead_time', 'ensemble', 'variable', 'correlation']
    percentiles : list of int
        Percentiles to compute across ensemble members.
        Must include 33 and 66 for skill classification.

    Returns
    -------
    summary_df : pd.DataFrame
        Percentile summary for each (start_month, lead_time, variable) group.
        Columns: ['start_month','lead_time','variable','p33','p50','p66', ...]
    classified_df : pd.DataFrame
        Original dataframe with added percentile columns and a 'category' column
        labeling member skill as 'problematic', 'acceptable', or 'good'.
    """

    # ---- Step 1: Compute percentiles across ensemble members ----
    # Group by start month, lead time, and variable
    grouped = df.groupby(['start_month','lead_time','variable'])

    # Compute requested percentiles (e.g., 33rd, 50th, 66th) for each group
    pct = grouped['correlation'].quantile([p/100 for p in percentiles]).unstack()

    # Rename columns to indicate percentile (e.g., p33, p50, p66)
    pct.columns = [f"p{p}" for p in percentiles]

    # Convert MultiIndex to columns for easy merging later
    summary_df = pct.reset_index()

    # ---- Step 2: Merge percentiles back into the original dataframe ----
    # This ensures every ensemble member has its group's percentile values
    classified_df = df.merge(summary_df, on=['start_month','lead_time','variable'], how='left')

    # ---- Step 3: Classify each ensemble member based on percentiles ----
    def classify(row):
        """
        Assign skill category based on correlation relative to 33rd and 66th percentiles.
        """
        if row['correlation'] < row['p33']:
            return "problematic"
        elif row['correlation'] < row['p66']:
            return "acceptable"
        else:
            return "good"

    # Apply classification row-wise
    classified_df['category'] = classified_df.apply(classify, axis=1)

    # ---- Step 4: Return results ----
    return summary_df, classified_df


In [9]:
summary_df, classified_df = summarize_with_brackets(anomaly_corr)
classified_df.to_csv("output/classified_df.csv", index=False)
summary_df.to_csv("output/summary_df.csv", index=False)
classified_df

,start_month,lead_time,ensemble,variable,correlation,p33,p50,p66,category
0,1,0,0,T2MAX,-0.313361,-0.297682,-0.138657,-0.007099,problematic
1,1,0,0,T2MIN,-0.067518,0.026461,0.089179,0.173905,problematic
2,1,0,1,T2MAX,-0.257885,-0.297682,-0.138657,-0.007099,acceptable
3,1,0,1,T2MIN,0.113398,0.026461,0.089179,0.173905,acceptable
4,1,0,2,T2MAX,0.339319,-0.297682,-0.138657,-0.007099,good
...,...,...,...,...,...,...,...,...,...
5503,9,5,48,T2MIN,0.267736,-0.270240,0.166433,0.370947,acceptable
5504,9,5,49,T2MAX,-0.260775,-0.258055,-0.061331,0.263059,problematic
5505,9,5,49,T2MIN,-0.050953,-0.270240,0.166433,0.370947,acceptable
5506,9,5,50,T2MAX,0.221388,-0.258055,-0.061331,0.263059,acceptable
